## Read .ghg metadata

In [42]:
import os
import time
import pandas as pd
from Parse_GHG import read_GHG
from ipywidgets import FloatProgress
from IPython.display import display

Dpath = 'C:\\highfreq\\'
Site = 'BB2'
raw_dir = Dpath+Site+'\\raw\\'
meta_dir = Dpath+Site+'\\metadata\\'

# Values we expect to change
dynamicMetaData_Headers = ['altitude','canopy_height']

# Values we want to fix
staticMetaData_FloatHeaders = ['altitude', 'latitude', 'longitude']
staticMetaData_TextHeaders = ['site_name']

T1 = time.time()
i = 0
print('Processing ghg files ')
f = FloatProgress(min=0, max=1) 
display(f) 
# Walk through the directory to find all "raw" folders
for (root, dirs, files) in sorted(os.walk(raw_dir)):
    # if root.__contains__('raw') :
    for file in files:
        name, tag = file.split('.')
         # .ghg files are located at the end of each directory tree
         # Avoids reading any that might be misplaced elsewhere
        if tag == 'ghg' and len(dirs)==0:  
            # read info in .ghg files and add to a dataframe
            if i == 0:
                GHG_out = read_GHG(root,name)
                Records = GHG_out.Summary.copy()
                config_template = GHG_out.config   
                i += 1
            elif i <= 5:#48*3:
                GHG_out = read_GHG(root,name)
                Records = pd.concat(
                    [Records,GHG_out.Summary],
                axis=0,
                ignore_index=True
                )
                i += 1
        f.value = i/len(files)

print('Processing time ', time.time()-T1) 
print('To inspect ', i, ' .ghg files')

staticMetaData = Records[staticMetaData_FloatHeaders].astype(float).mean()
staticMetaData = pd.concat([Records[staticMetaData_TextHeaders].astype(str).mode().T[0],staticMetaData])
print(staticMetaData)

dynamicMetaData = Records.set_index('TimeStamp')[dynamicMetaData_Headers]
dynamicMetaData['date']=dynamicMetaData.index.date
dynamicMetaData['time']=dynamicMetaData.index.time
dynamicMetaData.to_csv(meta_dir+'dynamicMetaData.csv',index=False)
dynamicMetaData


Processing ghg files 


FloatProgress(value=0.0, max=1.0)

Processing time  2.16691517829895
To inspect  6  .ghg files
Attribute
site_name     Burns Bog
altitude      10.166667
latitude      49.119003
longitude   -122.995094
dtype: object


Attribute,altitude,canopy_height,date,time
TimeStamp,,,,
2022-05-19 11:35:30,4.6,0.65,2022-05-19,11:35:30
2022-05-19 12:00:00,4.6,0.65,2022-05-19,12:00:00
2022-05-19 12:30:00,8.3,0.65,2022-05-19,12:30:00
2022-05-19 13:00:00,8.3,0.65,2022-05-19,13:00:00
2022-05-19 13:30:00,17.6,0.65,2022-05-19,13:30:00
2022-05-19 14:00:00,17.6,0.65,2022-05-19,14:00:00
